In [1]:
import numpy as np 
import scipy.stats as sps
import pandas as pd
import h5py
import os
from tqdm import tqdm

from copy import deepcopy

from numpy import array
from os import environ
from os.path import join
from sys import argv

from glob import glob
from numpy import zeros
from os.path import basename, join

from keras.models import load_model
from os import environ
from os.path import abspath, dirname, join

from copy import copy

from sklearn.cross_validation import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.
/Users/sergmiller/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/sergmiller/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import keras
from keras import backend as K

from keras.models import Sequential
from keras.models import model_from_json

from keras.layers import InputLayer, Input, Dense, Activation, Dropout, BatchNormalization, Reshape
from keras.layers import Average, Conv2D, MaxPooling2D, Flatten
from keras.layers.merge import Concatenate
from keras.activations import relu, softmax, sigmoid
from keras.layers.advanced_activations import LeakyReLU
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Model

from keras.regularizers import l2

from keras.constraints import maxnorm

from keras.optimizers import SGD, Adam
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical

from keras.losses import binary_crossentropy, mse

from keras.callbacks import LearningRateScheduler

In [51]:
import datagen_Miller
import models.keras_lstm
import models.dnn
import feature_generator

In [4]:
from importlib import reload

In [56]:
def run_model(X_train, y_train, X_val, y_val, 
          bad_columns=None,
          batch_size=1000,
          feature_expander=None,
          forecast_win=10,
          features_win=100,
          epochs=50,
          input_shape=None,
          path_to_load=None, 
          path_to_save=None,
          isLstm=False,
         ):
    
    assert isinstance(X_train, pd.DataFrame)
    assert input_shape is not None
    
    if path_to_load is None:
        model = models.keras_lstm.keras_lstm_model_1(max_sequence_length=batch_size,
                                              input_shape=input_shape)
#         model = models.dnn.nn_1(input_shape=80)
    else:
        model = load_model(path_to_load)
        optimizer = Adam(1e-3, clipvalue=0.5)
        model.compile(loss='mean_squared_error', 
                      optimizer=optimizer)
    
    datagen = datagen_Miller.DataGenerator(
                                        batch_size,
                                        feature_expander,
                                        forecast_win, 
                                        features_win,
                                        isLstm,
                                          )
    datagen.fit()
    
    filepath = "weights.{epoch:02d}-{val_loss:.2f}.hdf5"
    
    print('val steps {}'.format(X_val.shape[0] // batch_size))
    
    model.fit_generator(datagen.flow(X_train, y_train),
                        steps_per_epoch=X_train.shape[0] / batch_size, 
                        epochs=epochs, 
                        shuffle=False,
        callbacks=[
            datagen_Miller.TelegramCallback(),
            keras.callbacks.ModelCheckpoint(filepath, 
                                        monitor='val_loss', 
                                        verbose=0, 
                                        save_best_only=False, 
                                        save_weights_only=False, 
                                        mode='auto', 
                                        period=1)
                ],
        validation_data=datagen.flow(X_val, y_val),
        validation_steps=X_val.shape[0] // batch_size,
                       )
    
    if path_to_save:
        model.save(path_to_save)

In [74]:
reload(datagen_Miller)
reload(models.keras_lstm)
reload(models.dnn)
reload(feature_generator)

<module 'feature_generator' from '/Users/sergmiller/Documents/code/python/NightPy/feature_generator.py'>

In [7]:
%%time
# X_train = pd.read_csv('data/splits/tea_full_train.csv')
val = pd.read_csv('data/splits/tea_full_validation.csv')
# X_test = pd.read_csv('data/splits/tea_full_test.csv')

CPU times: user 13.7 s, sys: 1.25 s, total: 14.9 s
Wall time: 15.5 s


In [8]:
val.shape

(1567308, 44)

In [9]:
X_train = val.iloc[:1000]
X_val = val.iloc[1000:2000]

In [10]:
filter_columns = ['Unnamed: 0', '0_ID', 'Y', '1_TIME']

In [11]:
y_train = np.array(X_train['Y'])
X_train = pd.DataFrame(X_train.drop(filter_columns, axis=1), dtype=np.float32)
y_val = np.array(X_val['Y'])
X_val = pd.DataFrame(X_val.drop(filter_columns, axis=1), dtype=np.float32)

In [106]:
# for col in X_train:
#     print(type(X_train[col].iloc[0]))

In [55]:
X_train.shape

(1000, 40)

In [77]:
%%time
run_model(X_train, y_train, X_val, y_val, batch_size=20,
          feature_expander=feature_generator.count_all_features,
          input_shape=262,
          features_win=2,forecast_win=2, epochs=10, path_to_save='init_model', isLstm=True)

val steps 50
Epoch 1/10
33/50 [==================>...........] - ETA: 3s - loss: 0.0287- ETA: 7s - loss: - ETA: 5s - loss:

/Users/sergmiller/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:116: UserWarning: Method on_batch_end() is slow compared to the batch update (0.102513). Check your callbacks.
  % delta_t_median)


37/50 [=====================>........] - ETA: 2s - loss: 0.0261- ETA: 3s - loss: 0.026

/Users/sergmiller/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:116: UserWarning: Method on_batch_end() is slow compared to the batch update (0.103685). Check your callbacks.
  % delta_t_median)


46/50 [==========================>...] - ETA: 0s - loss: 0.0259- ETA: 1s - loss: 0.02

KeyboardInterrupt: 

In [33]:
X_train.head()

,Unnamed: 0,0_ID,1_TIME,BID_P_1,BID_V_1,BID_P_2,BID_V_2,BID_P_3,BID_V_3,BID_P_4,...,ASK_P_6,ASK_V_6,ASK_P_7,ASK_V_7,ASK_P_8,ASK_V_8,ASK_P_9,ASK_V_9,ASK_P_10,ASK_V_10
0,5617875,TEA,1970.01.02 17:59:17.691677,2561,81,2560,73,2559,99,2558,...,2567,109.0,2568.0,92.0,2569.0,105.0,2570.0,77.0,2571.0,124.0
1,5617876,TEA,1970.01.02 17:59:17.692349,2561,81,2560,73,2559,99,2558,...,2567,109.0,2568.0,92.0,2569.0,105.0,2570.0,77.0,2571.0,124.0
2,5617877,TEA,1970.01.02 17:59:17.692369,2561,81,2560,73,2559,99,2558,...,2567,109.0,2568.0,92.0,2569.0,105.0,2570.0,77.0,2571.0,124.0
3,5617878,TEA,1970.01.02 17:59:17.692587,2561,81,2560,73,2559,99,2558,...,2567,109.0,2568.0,92.0,2569.0,105.0,2570.0,77.0,2571.0,124.0
4,5617879,TEA,1970.01.02 17:59:17.692607,2561,81,2560,73,2559,99,2558,...,2567,109.0,2568.0,92.0,2569.0,104.0,2570.0,77.0,2571.0,124.0
